In [3]:
from ast import literal_eval
from collections import defaultdict
import pandas as pd
from pymatgen.core import Structure
from pymatgen.analysis.structure_matcher import StructureMatcher
data = pd.read_csv('../generated/WyFormer-DiffCSP++-10k.csv.gz', index_col="material_id",
                   converters={"structure": lambda s: Structure.from_dict(literal_eval(s))})

In [7]:
def filter_by_unique_structure_chem_sys_index(data: pd.DataFrame) -> pd.DataFrame:
    present = defaultdict(list)
    unique_indices = []
    for index, structure in data.structure.items():
        # Strutures consisiting of different sets of elements
        # can't match in any way
        chem_system = frozenset(structure.composition)
        if chem_system not in present:
            unique_indices.append(index)
        else:
            for present_structure in present[chem_system]:
                if StructureMatcher(attempt_supercell=True).fit(structure, present_structure, symmetric=True):
                    break
            else:
                unique_indices.append(index)
        present[chem_system].append(structure)
    return data.loc[unique_indices]

In [8]:
unique = filter_by_unique_structure_chem_sys_index(data)

In [9]:
len(unique) / len(data)

0.9874259586407566